##### This script can be used to align z profiles of one channel (identifyer: ch1_) against another channel (identifyer: ch2_). It was used in the paper Twamley et al. to align  LDL and WGA z profiles to the peak of the nuclear signal from dual color images.

In [ ]:
import pandas as pd
import numpy as np
from os import path
from os import makedirs
import os
import scipy.stats
import matplotlib.pyplot as plt
from scipy import signal

### enter folder name here:

In [ ]:
path="............."
CSVpath=path+"Profiles/"
CSVfiles=os.listdir(CSVpath)

In [ ]:
def split(matrix,identifyer,axis):
    splitted=matrix.filter(like=identifyer,axis=axis)
    return splitted

### enter identifiers for your conditions here:

In [ ]:
conditions=["Flow","Static"]

In [ ]:
values=[]
CSVs_=[]
for f in CSVfiles:
    tmp=[]
    CSVs_+=[f]
    table=pd.read_csv(CSVpath+f)
    tmp=table["Mean"].values.tolist()
    values+=[tmp]

In [ ]:
summary=pd.DataFrame(values).T
summary.columns=CSVs_
summary.to_csv(path+"Profile_summary_raw.csv")

In [ ]:
avDapi=split(summary,"ch1_",1) # only Dapi averages
avchTwo=summary[[a.replace("ch1_", "ch2_") for a in avDapi.columns.tolist()]]

indexList=[]
corDapi=[]
corchTwo=[]
for l in range(0,avDapi.shape[1]):
    trace=avDapi[avDapi.columns[l]].dropna().values
    s_trace=trace
    indexList+=[np.argmax(s_trace)]

for l in range(0,avDapi.shape[1]): 
    addMe=[np.NAN]*(300-indexList[l])
    newDapi=np.insert(avDapi[avDapi.columns[l]].values,0,addMe).tolist()
    newchTwo=np.insert(avchTwo[avchTwo.columns[l]].values,0,addMe).tolist()
    corDapi+=[newDapi]
    corchTwo+=[newchTwo]
corDapi=pd.DataFrame(corDapi).T
corDapi.columns=avDapi.columns
corchTwo=pd.DataFrame(corchTwo).T
corchTwo.columns=avchTwo.columns

corDapi.to_csv(path+"Profile_summary_corrected_Dapi.csv")
corchTwo.to_csv(path+"Profile_summary_corrected_chTwo.csv")

In [ ]:
allMeans=[]
allSEMs=[]
allSDs=[]
for c in conditions:
    allMeans+=[np.nanmean(split(corDapi,c,1),axis=1).tolist()]
    allSEMs+=[scipy.stats.sem(split(corDapi,c,1).values,nan_policy='omit').tolist()]
    allSDs+=[np.nanstd(split(corDapi,c,1),axis=1).tolist()]
allMeansDapi=pd.DataFrame(allMeans).T
allSEMsDapi=pd.DataFrame(allSEMs).T
allSDsDapi=pd.DataFrame(allSDs).T
allMeansDapi.columns=allSEMsDapi.columns=allSDsDapi.columns=conditions


allMeans=[]
allSEMs=[]
allSDs=[]
for c in conditions:
    allMeans+=[np.nanmean(split(corchTwo,c,1),axis=1).tolist()]
    allSEMs+=[scipy.stats.sem(split(corchTwo,c,1).values,nan_policy='omit').tolist()]
    allSDs+=[np.nanstd(split(corchTwo,c,1),axis=1).tolist()]
allMeanschTwo=pd.DataFrame(allMeans).T
allSEMschTwo=pd.DataFrame(allSEMs).T
allSDschTwo=pd.DataFrame(allSDs).T
allMeanschTwo.columns=allSEMschTwo.columns=allSDschTwo.columns=conditions

In [ ]:
allMeanschTwo.to_excel(path+"Profile_chTwo_Means.xls")
allSDschTwo.to_excel(path+"Profile_chTwo_SDs.xls")
allMeansDapi.to_excel(path+"Profile_Dapi_Means.xls")
allSDsDapi.to_excel(path+"Profile_Dapi_SDs.xls")